In [8]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
#table_name = 'YOUR_SNOWFLAKE_TABLE_NAME'

#sf_df = my_session.sql("select * from {}".format(table_name))
#df = sf_df.to_pandas()

In [4]:
df = pd.read_csv('Attendance_dataset.csv')

In [6]:
df.shape

(1000, 5)

In [10]:
df['Attendance_Date'] = pd.to_datetime(df['Attendance_Date'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Employee_Id      1000 non-null   object        
 1   Employee_name    1000 non-null   object        
 2   Attendance_Date  1000 non-null   datetime64[ns]
 3   Present_Status   1000 non-null   object        
 4   Hours_Worked     1000 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 39.2+ KB


In [13]:
df.describe()

,Attendance_Date,Hours_Worked
count,1000,1000.000000
mean,2023-02-14 13:00:28.799999744,2.253000
min,2023-01-01 00:00:00,0.000000
25%,2023-01-24 00:00:00,0.000000
50%,2023-02-15 00:00:00,0.000000
75%,2023-03-07 00:00:00,5.000000
max,2023-03-31 00:00:00,8.000000
std,NaN,2.846593


In [14]:
df.isnull().sum()

Employee_Id        0
Employee_name      0
Attendance_Date    0
Present_Status     0
Hours_Worked       0
dtype: int64